In [18]:
import pymc3 as pm
import scipy
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

theano.config.compute_test_value = 'raise'
#theano.config.gcc.cxxflags = "-fbracket-depth=16000" # default is 256

def load_data():
    temp_expected = pd.read_csv('~/4th Year/project/data/csv/expected.csv')
    E = np.array(temp_expected['x'])
    
    temp_sim = pd.read_csv('~/4th Year/project/data/csv/simulated.csv')
    temp_times = temp_sim[['Time1', 'Time2', 'Time3', 'Time4', 'Time5', 'Time6', 'Time7', 'Time8', 'Time9', 'Time10', 'Time11', 'Time12', 'Time13', 'Time14', 'Time15']]
    observed_values = np.matrix(temp_times)
    
    adj = pd.read_csv('/Users/Mike/4th Year/project/data/csv/adjacency.csv', index_col=0)
    W = np.matrix(adj)
    
    numRegions = observed_values.shape[0] #number of regions
    nt = observed_values.shape[1] #number of time points
    Q = np.diag(np.ones(nt-1), k=1) #'adjacency matrix' in time
    
    #making the inverse covariance matricies for the CAR models (ignoring their variances)
    alpha = 0.75 #this was 1 in the model but that makes the covariance matrix singular
    D = np.diag(np.array(W.sum(0))[0]) #diag(d_1,..,d_numRegions) with d_i the number of neighbours of region i
    Tau_v_unscaled = np.array(D - alpha*W)
    Tau_gamma_unscaled = np.identity(n=nt) - Q

    return numRegions, nt, E, Tau_v_unscaled, Tau_gamma_unscaled, observed_values

prob_z = 0.95 #probability of a region following the area specific model

numRegions, nt, E, Tau_v_unscaled, Tau_gamma_unscaled, observed_values = load_data()


In [21]:
model = pm.Model()

numRegions = 30
nt = 3

with model:
    
    spatial_sd = pm.Flat('spatial_sd')
    
    spatial_dev = np.empty(shape=numRegions, dtype=object)
    mu = np.empty(shape=(numRegions, nt), dtype=object)
    for i in range(numRegions):
        spatial_dev[i] = pm.HalfNormal('spatial_{}'.format(i), sd=spatial_sd)
        for t in range(nt):
            mu[i, t] = E[i]*spatial_dev[i]
        
    observed = np.empty(shape=(numRegions, nt), dtype=object)
    for i in range(numRegions):
        for t in range(nt):
            observed[i, t] = pm.Poisson('observed_{}_{}'.format(i,t), mu = mu[i,t], observed=observed_values[i,t])

INFO (theano.gof.compilelock): Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock


In [22]:
with model:
    start = pm.find_MAP()

INFO (theano.gof.compilelock): Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO (theano.gof.compilelock): Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO (theano.gof.compilelock): Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /Users/Mike/.theano/compiledir_Darwin-15.0.0-x86_64-i386-64bit-i386-3.5.2-64/lock_dir/lock


Optimization terminated successfully.
         Current function value: 10000000000000000159028911097599180468360808563945281389781327557747838772170381060813469985856815104.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


/Users/Mike/anaconda/lib/python3.5/site-packages/theano/gradient.py:537: UserWarning: grad method was asked to compute the gradient with respect to a variable that is not part of the computational graph of the cost, or is used only by a non-differentiable operator: spatial_sd
  handle_disconnected(elem)
/Users/Mike/anaconda/lib/python3.5/site-packages/theano/gradient.py:563: UserWarning: grad method was asked to compute the gradient with respect to a variable that is not part of the computational graph of the cost, or is used only by a non-differentiable operator: <DisconnectedType>
  handle_disconnected(rval[i])


ValueError: Optimization error: max, logp or dlogp at max have non-finite values. Some values may be outside of distribution support. max: {'spatial_4_log_': array(-inf), 'spatial_18_log_': array(-inf), 'spatial_13_log_': array(-inf), 'spatial_3_log_': array(-inf), 'spatial_21_log_': array(-inf), 'spatial_8_log_': array(-inf), 'spatial_10_log_': array(-inf), 'spatial_5_log_': array(-inf), 'spatial_2_log_': array(-inf), 'spatial_14_log_': array(-inf), 'spatial_23_log_': array(-inf), 'spatial_16_log_': array(-inf), 'spatial_29_log_': array(-inf), 'spatial_28_log_': array(-inf), 'spatial_6_log_': array(-inf), 'spatial_15_log_': array(-inf), 'spatial_26_log_': array(-inf), 'spatial_11_log_': array(-inf), 'spatial_9_log_': array(-inf), 'spatial_12_log_': array(-inf), 'spatial_22_log_': array(-inf), 'spatial_27_log_': array(-inf), 'spatial_25_log_': array(-inf), 'spatial_19_log_': array(-inf), 'spatial_1_log_': array(-inf), 'spatial_7_log_': array(-inf), 'spatial_sd': array(0.0), 'spatial_0_